In [1]:
import time
from datetime import timedelta

import html
import re

import os
import ntpath

import subprocess as sp

import numpy as np
import pandas as pd
import xml.etree.ElementTree as et

import dask
import dask.dataframe as dd
import dask.bag as bd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

## Read csv files Java Answers

In [2]:
df = dd.read_csv('checkstylexmlreports_csv/checkstyleReport*.csv', engine='python', error_bad_lines=False, warn_bad_lines=False, dtype=object)


## Start a Dask cluster using SLURM jobs as workers

In [3]:
#http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html
dask.config.set(
    {
        "distributed.worker.memory.target": False,  # avoid spilling to disk
        "distributed.worker.memory.spill": False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=2, #cores=24, # we set each job to have 1 Worker, each using 10 cores (threads) and 8 GB of memory
    processes=2,
    memory="8GiB",
    walltime="0-00:30",# walltime="0-00:50",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
)

Spawn between 20 to 100 workers and connect a client to be able use them.

In [4]:
#cluster.scale(n=20) # ask for 20 jobs or workers
# This also works with adaptive clusters. This automatically launches and kill workers based on load.
# we tell our cluster to autoscale between 10 and 20 workers depending on the load
cluster.adapt(minimum_jobs=10, maximum_jobs=200)
#cluster.adapt(maximum_jobs=20)
client = Client(cluster)
client

Client Scheduler: tcp://192.168.94.166:39317 Dashboard: http://192.168.94.166:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
df

,name,line,severity,message,source
npartitions=101,,,,,
,object,object,object,object,object
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [6]:
checkstyle_codeviolations_init_len = df.shape[0].compute()
print('Initial number of volations {}'.format(checkstyle_codeviolations_init_len))

Initial number of volations 8074616


In [7]:
# drop empty columns
df = df[~df['name'].isna()] # Drop rows that have NaN in the Code column

In [8]:
checkstyle_codeviolations_nonan_len =df.shape[0].compute()
print('Number of violations after removing the nans in the class column {}'.format(checkstyle_codeviolations_nonan_len))

Number of violations after removing the nans in the class column 8074616


In [9]:
df.shape[0].compute()

8074616

#### Get the unique classes dataframe

In [10]:
# get the unique classes
unique_classes_df = df['name'].drop_duplicates().compute()

In [11]:
# This also counts the unique classes
unique_classes_df.shape[0]

731116

### Get the unique Violations

In [12]:
grp_vio = df.groupby('source')

In [13]:
#count the unique rulesets
grp_vio.first().head()

,name,line,severity,message
source,,,,
com.puppycrawl.tools.checkstyle.checks.ArrayTypeStyleCheck,Code_30381834_6434168_4179_3.java,2,error,Array brackets at illegal position.
com.puppycrawl.tools.checkstyle.checks.FinalParametersCheck,Code_30381834_6434168_4179_3.java,2,error,Parameter args should be final.
com.puppycrawl.tools.checkstyle.checks.NewlineAtEndOfFileCheck,Code_30381834_6434168_4179_3.java,1,error,File does not end with a newline.
com.puppycrawl.tools.checkstyle.checks.TodoCommentCheck,Code_47202560_47190189_667_0.java,76,error,Comment matches to-do format 'TODO:'.
com.puppycrawl.tools.checkstyle.checks.UpperEllCheck,Code_54470504_54469628_699_1.java,7,error,Should use uppercase 'L'.


### Count the number of times each items occured

In [22]:
#https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
viol_count_df = pd.DataFrame(df.groupby('source')['source'].count().compute())
# rename the column to total
viol_count_df.columns=['total']
# drop the index level
viol_count_df = viol_count_df.reset_index()
viol_count_df

,source,total
0,com.puppycrawl.tools.checkstyle.checks.ArrayTy...,28859
1,com.puppycrawl.tools.checkstyle.checks.FinalPa...,863887
2,com.puppycrawl.tools.checkstyle.checks.Newline...,484311
3,com.puppycrawl.tools.checkstyle.checks.TodoCom...,1870
4,com.puppycrawl.tools.checkstyle.checks.UpperEl...,807
...,...,...
56,com.puppycrawl.tools.checkstyle.checks.whitesp...,186132
57,com.puppycrawl.tools.checkstyle.checks.whitesp...,988
58,com.puppycrawl.tools.checkstyle.checks.whitesp...,414293
59,com.puppycrawl.tools.checkstyle.checks.whitesp...,1038215


In [24]:
viol_count_df['percent'] = (viol_count_df['total']/ viol_count_df['total'].sum())*100

In [25]:
viol_count_df.head()

,source,total,percent
0,com.puppycrawl.tools.checkstyle.checks.ArrayTy...,28859,0.357404
1,com.puppycrawl.tools.checkstyle.checks.FinalPa...,863887,10.698800
2,com.puppycrawl.tools.checkstyle.checks.Newline...,484311,5.997945
3,com.puppycrawl.tools.checkstyle.checks.TodoCom...,1870,0.023159
4,com.puppycrawl.tools.checkstyle.checks.UpperEl...,807,0.009994
5,com.puppycrawl.tools.checkstyle.checks.blocks....,2401,0.029735
6,com.puppycrawl.tools.checkstyle.checks.blocks....,10013,0.124006
7,com.puppycrawl.tools.checkstyle.checks.blocks....,236167,2.924808
8,com.puppycrawl.tools.checkstyle.checks.blocks....,98816,1.223786
9,com.puppycrawl.tools.checkstyle.checks.blocks....,40155,0.497299


### Calculate number to be sampled per ruleset

In [26]:
import math
def comp_smpl_no(x):
    x = round((x/100)*384)
    if x < 1:
        x = 1
    return x

# Randomly select that number
#https://datatofish.com/random-rows-pandas-dataframe/
#chk_style_ran = df.sample(frac=384/len(df), replace=None, random_state=10)
viol_count_df['sample_size_384'] = viol_count_df['percent'].apply(comp_smpl_no)
viol_count_df.head()

,source,total,percent,sample_size_384
0,com.puppycrawl.tools.checkstyle.checks.ArrayTy...,28859,0.357404,1
1,com.puppycrawl.tools.checkstyle.checks.FinalPa...,863887,10.698800,41
2,com.puppycrawl.tools.checkstyle.checks.Newline...,484311,5.997945,23
3,com.puppycrawl.tools.checkstyle.checks.TodoCom...,1870,0.023159,1
4,com.puppycrawl.tools.checkstyle.checks.UpperEl...,807,0.009994,1


In [16]:
viol_count_df.head()

,name,line,severity,message,source
54163,Code_3033544_3033517_2908_0.java,7,error,Variable 'color' must be private and have acce...,com.puppycrawl.tools.checkstyle.checks.design....
14988,Code_28045744_28042594_928_0.java,3,error,Variable 'c' must be private and have accessor...,com.puppycrawl.tools.checkstyle.checks.design....
37273,Code_61372799_61372436_442_2.java,14,error,Missing a Javadoc comment.,com.puppycrawl.tools.checkstyle.checks.javadoc...
9127,Code_34542279_33601920_3454_0.java,7,error,Parameter generator should be final.,com.puppycrawl.tools.checkstyle.checks.FinalPa...


In [17]:
chk_style_ran['name'].shape[0].compute()

400

In [18]:
nn

NameError: name 'nn' is not defined

In [ ]:
## Make a folder in that directory
#folder = '{}/sample_pmd_csv'.format(common_path)
folder = 'sample_ckstyl'
# output: path/to/Post.csv => path/to
mkdir_cmd = 'mkdir {}'.format(folder)
cmd = sp.run(
    mkdir_cmd, # command
    capture_output=True,
    text=True,
    shell=True
)

In [ ]:
# copy list of sample java files into the destination 
for file_to_copy in chk_style_ran['name']:
    copy_file_cmd = 'cp codesnippets_java/{} {}'.format(file_to_copy, folder)
    cmd = sp.run(
        copy_file_cmd, # command
        capture_output=True,
        text=True,
        shell=True
    )

# Sort them in order

In [ ]:
chk_style_ran = chk_style_ran.sort_values('name')

In [ ]:
chk_style_ran.head()

In [ ]:
#for name, msg in zip(chk_style_ran['name'], chk_style_ran['message']):
#    print('Class: {} Message: {}'.format(name, msg))

In [ ]:
for name, ln, msg in zip(chk_style_ran['name'], chk_style_ran['line'], chk_style_ran['message']):
    print('Class: {}  Line: {} Message: {}'.format(name, ln, msg))

In [ ]:
'com.puppycrawl.tools.checkstyle.checks.design'.split('.')[-1]

In [ ]:
'com.puppycrawl.tools.checkstyle.checks.design'.split('.')[-2]